In [1]:
import sys
import numpy as np

In [2]:
def sigmoid(x):
    return 1.0 / (1.0 + np.exp(-x))

In [3]:
def sigmoid_deriv_from(sigm):
    return sigm * (1.0 - sigm)

In [4]:
def tanh(x):
    return np.tanh(x)
def tanh_deriv_from(tanh_x):
    return 1.0 - tanh_x ** 2

In [20]:
class NN:   
    def __init__(self, number_of_neurons_in_layers, activation_function=0, seed=None):
        if seed is not None:
            np.random.seed(seed)

        self.number_of_neurons_in_layers = number_of_neurons_in_layers
        self.activation_function = activation_function

        if activation_function == 0:
            self.activation_function = sigmoid
            self.activation_function_derivative = sigmoid_deriv_from
        elif activation_function == 1:
            self.activation_function = tanh
            self.activation_function_derivative = tanh_deriv_from
        self.Weights_list = []
        total_layers = len(number_of_neurons_in_layers)
        for i in range(total_layers - 1):
            neurons_in = number_of_neurons_in_layers[i]
            neurons_out = number_of_neurons_in_layers[i + 1]
            w = np.random.uniform(-0.5, 0.5, size=(neurons_out, neurons_in + 1))  
            self.Weights_list.append(w)

    def _add_bias(self, x_values):
        return np.concatenate([x_values, np.ones((1,))], axis=0)

    def forward_propagation(self, x):
        a = x.astype(float)
        activations_output = [a]
        neuron_output_zs = []

        for w in self.Weights_list:
            a__with_bias = self._add_bias(a)
            z = w @ a__with_bias
            a = self.activation_function(z)
            neuron_output_zs.append(z)
            activations_output.append(a)

        return activations_output, neuron_output_zs   # returns a list of the outputs of all the layers (and zs separately);
                                                      # each element of the activation_output is a vector of the neurons' outputs in that layer

    def predict(self, x):
           activations_output, zs = self.forward_propagation(x)
           return activations_output[-1]    #returns the output of the last layer of the network

    def train_network_with_backpropagation(self, X, y, learning_rate=0.1, K=10000):  #K is the number of epochs 
        for _ in range(K):
            for xi, searched_value in zip(X, y):
                activations_output, zs = self.forward_propagation(xi)
                deltas = [None] * len(self.Weights_list)

                activation_output_last_layer = activations_output[-1]
                deltas[-1] = (searched_value - activation_output_last_layer) * self.activation_function_derivative(activation_output_last_layer)

                for l in range(len(self.Weights_list) - 2, -1, -1):  #hidden layers backpropagation
                    w_next = self.Weights_list[l + 1]
                    w_next_without_bias = w_next[:, :-1]
                    a = activations_output[l + 1]
                    deltas[l] = (w_next_without_bias.T @ deltas[l + 1]) * self.activation_function_derivative(a)

                for l in range(len(self.Weights_list)):   #weights updating
                    a_prev = activations_output[l]
                    a_prev = self._add_bias(a_prev) 
                    self.Weights_list[l] += learning_rate * np.outer(deltas[l], a_prev)

In [6]:
def parse_data(boolean_operation, activation_function):
    X = np.array([
        [0, 0],
        [0, 1],
        [1, 0],
        [1, 1],
    ], dtype=float)

    if boolean_operation == "AND":
        y = np.array([[0], [0], [0], [1]], dtype=float)
    elif boolean_operation == "OR":
        y = np.array([[0], [1], [1], [1]], dtype=float)
    elif boolean_operation == "XOR":
        y = np.array([[0], [1], [1], [0]], dtype=float)
    if activation_function == 1:
        y = 2.0 * y - 1.0
    return X, y

In [7]:
def get_output_if_tanh(result):
    val = float(np.ravel(result)[0])
    val = (val + 1.0) / 2.0
    return val

In [8]:
def run_program(boolean_operation, activ_function, number_of_neurons_in_hidden_layers, learning_rate=0.1, K=10000, seed=1):
    X, y = parse_data(boolean_operation, activ_function)
    layer_sizes = [2] + number_of_neurons_in_hidden_layers + [1]
    
    network = NN(layer_sizes, activ_function, seed=seed)
    network.train_network_with_backpropagation(X, y, learning_rate, K)
    
    print(f"{boolean_operation}:")
    for xi in X:
        prediction = network.predict(xi)
        if activ_function == 1:
            result = get_output_if_tanh(prediction)
        else:
            result = float(np.ravel(prediction)[0])
        print(f"({int(xi[0])},{int(xi[1])}) -> {result:.4f}")
    print()


In [28]:
def read_input():
    boolean_operation = input().strip().upper()
    activation_function = int(input().strip())
    n_hidden = int(input().strip())

    neurons = []
    for _ in range(n_hidden):
        neurons.append(int(input().strip()))  

    if boolean_operation == "ALL":
        for op in ("AND", "OR", "XOR"):
            run_program(op, activation_function, neurons, learning_rate=0.1, K=100000, seed=1)
    else:
        run_program(boolean_operation, activation_function, neurons, learning_rate=0.1, K=10000, seed=1)

In [29]:
read_input()

AND:
(0,0) -> 0.0001
(0,1) -> 0.0066
(1,0) -> 0.0068
(1,1) -> 0.9892

OR:
(0,0) -> 0.0079
(0,1) -> 0.9946
(1,0) -> 0.9947
(1,1) -> 0.9994

XOR:
(0,0) -> 0.0105
(0,1) -> 0.9899
(1,0) -> 0.9900
(1,1) -> 0.0101

